## **2. RESNET Implementation**
### **Import necessary libraries** 

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

### **Download CIFAR-100 dataset**



In [0]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.26733, 0.2564, 0.2762))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5089, 0.4874, 0.4419), (0.2683, 0.2574, 0.2771))
])

trainset = torchvision.datasets.CIFAR100(root='./data/cifar100', train=True, download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR100(root='./data/cifar100', train=False, download=True, transform=transform_test)

trainloader= torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


### **Implement basic block & bottleneck block**

In [0]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [0]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

### **Build ResNet using predefined blocks**

In [0]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [0]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])

### **Implement train & test functions**

In [0]:
def train(model, trainloader, criterion, optimizer):    
    train_loss = 0.0
    train_acc = 0.0
    total = 0 
    correct = 0 
    
    train_loader = trainloader
    model.train()

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        output = model(inputs) 
        loss = criterion(output, labels)
        
        _, preds = output.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_acc = (correct / total) * 100.0 
        
    return (train_loss / (batch_idx + 1)) , train_acc

In [0]:
def test(model, testloader, criterion):
    test_loss = 0.0
    test_acc = 0.0
    total = 0
    correct = 0 
    
    test_loader = testloader
    model.eval()
    with torch.no_grad():
      for batch_idx, (inputs, labels) in enumerate(test_loader):
          inputs = inputs.to(device)
          labels = labels.to(device)
          
          optimizer.zero_grad()
          
          output = model(inputs)
          loss = criterion(output, labels)
          
          _, preds = output.max(1)
          total += labels.size(0)
          correct += preds.eq(labels).sum().item()
          
          test_loss += loss.item()
          test_acc = (correct / total) * 100.0
        
    return (test_loss / (batch_idx +1)), test_acc


### **Define the model and hyperparameters**

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

model = ResNet18()

num_epochs = 10

model = model.to(device)
params_to_update = model.parameters()
optimizer = optim.SGD(params_to_update, lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

### **Training loop**

In [0]:
for epoch in range(num_epochs):    
    train_loss, train_acc = train(model, trainloader, criterion, optimizer)
    if epoch % 1 == 0:
        test_loss, test_acc = test(model, testloader, criterion)
        print(f'Train accuracy: {train_acc} / Test accuracy: {test_acc}')

if os.path.isdir('./saved_models') == False:
    os.makedirs('./saved_models')

torch.save(model, './saved_models/resnet.pt')

Train accuracy: 13.25 / Test accuracy: 21.95
Train accuracy: 25.363999999999997 / Test accuracy: 29.080000000000002
Train accuracy: 33.822 / Test accuracy: 34.74
Train accuracy: 40.833999999999996 / Test accuracy: 41.68
Train accuracy: 46.152 / Test accuracy: 45.93
Train accuracy: 50.638000000000005 / Test accuracy: 47.61
Train accuracy: 53.181999999999995 / Test accuracy: 52.65
Train accuracy: 56.506 / Test accuracy: 50.56
Train accuracy: 58.504 / Test accuracy: 56.28999999999999
Train accuracy: 61.106 / Test accuracy: 58.03


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type BasicBlock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
